# Objective
Simulate the rotor allocation for quadrotor in the event of single rotor failure (in eq.16) in arducopter SITL using Dronekit API

In [ ]:
# Mavproxy command for running udp <--> udp connection
# mavproxy.exe --master tcp:127.0.0.1:5760 --sitl 127.0.0.1:5501 --out 127.0.0.1:14550 --out 127.0.0.1:14551 --baudrate 921600

In [ ]:
# Mavproxy command for running com port <--> udp connection
# Windows
# mavproxy.exe --master com3 --sitl 127.0.0.1:5501 --out 127.0.0.1:14550 --out 127.0.0.1:14551 --baudrate 921600
# Linux
# mavproxy.exe --master \dev\ttyUSB0,57600 --sitl 127.0.0.1:5501 --out 127.0.0.1:14550 --out 127.0.0.1:14551 --baudrate 921600

### What the problem is
- Multiple issues/PRs related to "controlling a single motor with PWM value/ speed range"
    - https://github.com/ArduPilot/ardupilot/issues/9796 - Open
    - https://github.com/PX4/PX4-Autopilot/pull/13240 - Open
    - https://github.com/dronekit/dronekit-sitl/issues/139 - Open
    - the list goes on.
- Existing mavlink command MAV_CMD_DO_SET_SERVO **does not work** when the drone is controlled by the PX4 controller.
    - i.e. it throws an error when you try the command on motors 1,2,3,4 (in case of quadcopter)
- Need to find a solution without playing too much with the firmware or without sacrificing a lot of ArduPilot functionalities

### How the problem is solved/ 1-motor failure is simulated
- Hacky solution was recently listed here (~ October 2021) - https://discuss.bluerobotics.com/t/how-to-control-thrusters-independently/9870/11
- Some versions of firmware (ArduPilot's newer ones specifically) have **parameters** for changing what the Servo output does.
    - e.g. SERVO1_FUNCTION can be set to
        - 0 for Disabled - meaning it will be controlled by the PX4 normally
        - 1 for RCPassThru - meaning internal controller will be skipped, but joystick input will be directly mapped to motor PWM
        - and so on
- Setting the parameter SERVO1_FUNCTION to 1 (RCPassThru) and then using MAV_CMD_DO_SET_SERVO works fine

### Cons of using this method:
- Complete control removed from controller - so you can't send a command from QGC/Mission Planner.
- Can only send command manually through our code.
- Parameters (such as SERVO1_FUNCTION) are stored inside EEPROM.
    - Parameters are loaded after device restart from EEPROM, so setting them once is setting them persistently.
    - This means, no arming/disarming sequence is required.
    - If you forget the reset the parameter value before shutting device down, and you accidentally touch the joystick the next time you are using the device, it will start moving, without any arming/disarming.

### Tests done/ what doesn't work
- dir(esalexander) - sample code provided for Autopilot override sequence using hack mentioned above. Code has failsafes and is only for demonstration for a few seconds. It does work. I have implemented the bare minimum similar code from this.
- test_rc_passthru - bare bones implementation of above hack. **This one works.**
- dir(pymavlink) - used for reading parameter values and writing them. Useful since many firmwares **did not** have the required parameters. No need to connect over UI. Can headlessly test over firmware version by running in SITL and checking if it has reqd params.
- manual_control - mavsdk demo for manual control. Useless.
- quadrotor_vars - dummy code.
- test_failure - mavsdk has capability to inject failure. But failure fails in case it is done on a motor that is already in use by the PX4 (1,2,3,4 in our case)
- test_mavproxy - test to see if MAV_CMD_DO_SET_SERVO works in a long_command mavlink message instead of a direct set_servo message. Error given is same, since motor is already in use.
- test1 - test for motor testing commmand MAV_CMD_DO_MOTOR_TEST. Fails since motor is already in use. Can only be used from QGC GUI, where a lock is set in before testing a single motor. Useless because it only lets you test one single motor, and you cannot use it on multiple at the same time
- test2 - basic RC override test. Fails because RC overrides functions actually map the roll, pitch, yaw, ... angles instead of the actual motor PWM 1, PWM 2, PWM 3, ... So directly overriding gives a wild behaviour, since, for e.g., roll value mapped to single motor.
- test3 - attitude setting test using quaternion. Useless since it still uses roll, pitch, yaw input values.
- test4 - copy of pymavlink test
- test5 - dummy code

### Dummy code/function to test eq(16) motor PWM values to Uf, Tq, Tr (instantaneous):

In [6]:
from scipy.interpolate import interp1d
import numpy as np

pwm_1 = 1800
pwm_3 = 1800
pwm_4 = 1800

## Mappings for thrust to force
## Used a sample thruster/motor for force calculation:
## https://uav-en.tmotor.com/html/2018/u_0222/3.html
## Prop size: 12*4CF
# 50% - 350 G
# 65% - 550 G
# 75% - 700 G
# 85% - 870 G
# 100% - 1000 G
x =  [1000, 1500, 1650, 1750, 1850, 2000]
y = [0, 350, 550, 700, 870, 1000]

# Function that maps motor PWM to Force in grams
pwm_to_force = interp1d(x, y, 'cubic')

## Since motor uses 12inch prop, min. required arm length = root(2) * half (arm length) = 1.414 * 100 cm - 30 > 0
## arm length = 100 cm gives ~10cm + space
# Arm Length (l)
l = arm_length = 1
# Drag to Thrust co-efficients of the blade (12*4CF)
d = drag_thrust_co_efficient = 4

left_matrix = np.array([[1, 1, 1], [-l, l, 0], [d, d, -d]])
right_matrix = np.array([pwm_to_force(pwm_1), pwm_to_force(pwm_3), pwm_to_force(pwm_4)])
force_matrix = np.dot(left_matrix,right_matrix)

print(force_matrix)

uf = force_matrix[0]
tq = force_matrix[1]
tr = force_matrix[2]

print("uf: ", uf)
print("tq: ", tq)
print("tr: ", tr)

[2356.94144661    0.         3142.58859548]
uf:  2356.941446613089
tq:  0.0
tr:  3142.5885954841183
